# Imports

In [1]:
!pip install spacy
!python -m spacy download en_core_web_sm
#from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, make_scorer
import pandas as pd 
import string
import re
import numpy as np
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.inspection import permutation_importance
from sklearn.model_selection import ParameterGrid
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
import pandas as pd
import string
import re
import numpy as np
from scipy.stats import lognorm
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
import spacy

# Load spaCy model for lemmatization
nlp = spacy.load("en_core_web_sm")



     ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
     ---- ----------------------------------- 1.6/12.8 MB 13.9 MB/s eta 0:00:01
     ------------- -------------------------- 4.5/12.8 MB 13.4 MB/s eta 0:00:01
     --------------------- ------------------ 6.8/12.8 MB 12.3 MB/s eta 0:00:01
     ------------------------- -------------- 8.1/12.8 MB 12.3 MB/s eta 0:00:01
     --------------------------------- ----- 11.0/12.8 MB 11.3 MB/s eta 0:00:01
     --------------------------------------  12.6/12.8 MB 11.3 MB/s eta 0:00:01
     --------------------------------------  12.6/12.8 MB 11.3 MB/s eta 0:00:01
     ---------------------------------------- 12.8/12.8 MB 8.3 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


# Unigram

## Dataset Preprocessing

In [2]:
traindf = pd.read_csv("train_reviews.csv")
trainsentences = traindf["Review"].values
trainlabels = traindf["Label"].values
testdf = pd.read_csv("test_reviews.csv")
testsentences = testdf["Review"].values
testlabels = testdf["Label"].values 

# Load stop words
with open("english", "r") as file:
    stop_words = set(file.read().splitlines())
with open("hotel_names.csv", "r") as file:
    hotel_names = set(file.read().splitlines())

# Preprocessing function with lemmatization
def preprocess_text(text):
    text = re.sub(r'\d+', '', text)
    text = text.translate(str.maketrans('', '', string.punctuation))
    text = text.lower()
    words = text.split()
    words = [word for word in words if word not in stop_words]
    words = [word for word in words if word not in hotel_names]
    text = ' '.join(words).strip()
    # Lemmatize text using spaCy
    # doc = nlp(" ".join(words))
    # lemmatized_text = " ".join([token.lemma_ for token in doc if not token.is_punct and token.lemma_ not in stop_words and token.lemma_ not in hotel_names])
    
    return text

train_sentences_preprocessed = [preprocess_text(sentence) for sentence in trainsentences]
test_sentences_preprocessed = [preprocess_text(sentence) for sentence in testsentences]
train_df_preprocessed = pd.DataFrame({'Review': train_sentences_preprocessed,'Label': trainlabels})
test_df_preprocessed = pd.DataFrame({'Review': test_sentences_preprocessed,'Label': testlabels})

In [3]:
trainsentences = train_df_preprocessed["Review"].values
y_train = train_df_preprocessed["Label"].values

testsentences = test_df_preprocessed["Review"].values
y_test = test_df_preprocessed["Label"].values

# Encode labels as integers
le = LabelEncoder()
y_train = le.fit_transform(y_train)
y_test = le.transform(y_test)

# vectorizing
vectorizer = CountVectorizer(ngram_range=(1,1)) # ngram range for specifying unigrams and bigrams (1,1) - unigram, (2,2) - bigram, (1,2) - both
X_train = vectorizer.fit_transform(trainsentences)
X_test = vectorizer.transform(testsentences)
vect2= TfidfVectorizer(ngram_range=(1,1))
X_train2 = vect2.fit_transform(trainsentences)
X_test2 = vect2.transform(testsentences)


## Multinomial naive Bayes

### Train and accuracy without feature selection

In [4]:
# parameters
param_grid = {
    'alpha': [0.1, 0.5, 1.0, 1.5, 2.0],  # Laplace smoothing parameter
    'fit_prior': [True, False]
}
multinomial_naive_bayes = MultinomialNB()

# Set up GridSearchCV
grid_search = GridSearchCV(multinomial_naive_bayes, param_grid, cv=5, scoring='accuracy')
grid_search.fit(X_train, y_train)

# Best parameters and score
print("Best Parameters:", grid_search.best_params_)
print("Best Cross-Validation Score:", grid_search.best_score_)

# Evaluate on test set
test_score = grid_search.score(X_test, y_test)
print("Test Set Score:", test_score)

Best Parameters: {'alpha': 0.5, 'fit_prior': True}
Best Cross-Validation Score: 0.81875
Test Set Score: 0.85625


In [5]:
# fit the model
multinomial_naive_bayes = MultinomialNB( alpha = 1.5, fit_prior = True)
multinomial_naive_bayes.fit(X_train, y_train)
# predictions
y_pred = multinomial_naive_bayes.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='binary')
recall = recall_score(y_test, y_pred, average='binary')
f1 = f1_score(y_test, y_pred, average='binary')

print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")

Accuracy: 0.8500
Precision: 0.9118
Recall: 0.7750
F1 Score: 0.8378


### Train and accuracy with chi squared test for feature selection

In [6]:
# Perform Chi-squared feature selection
chi2_selector = SelectKBest(chi2, k=2000) #2000 features perform best
X_train_chi2 = chi2_selector.fit_transform(X_train, y_train)
X_test_chi2 = chi2_selector.transform(X_test)

# Train a Multinomial Naive Bayes model
nb_model = MultinomialNB()
nb_model.fit(X_train_chi2, y_train)

# Predict on the test set
y_pred = nb_model.predict(X_test_chi2)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)

# Get the feature names for the selected features
selected_feature_indices = chi2_selector.get_support(indices=True)
selected_feature_names = vectorizer.get_feature_names_out()[selected_feature_indices]
precision = precision_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')
f1 = f1_score(y_test, y_pred, average='weighted')

print(f"Selected Features: {selected_feature_names}")
print(f"Test Set Accuracy: {accuracy}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")

Selected Features: ['abassador' 'accept' 'accepted' ... 'yunan' 'zone' 'zoo']
Test Set Accuracy: 0.85625
Precision: 0.8608
Recall: 0.8562
F1 Score: 0.8558


In [7]:
# parameters
param_grid = {
    'alpha': [0.1, 0.5, 1.0, 1.5, 2.0],  # Laplace smoothing parameter
    'fit_prior': [True, False]
}
multinomial_naive_bayes = MultinomialNB()

# Set up GridSearchCV
grid_search = GridSearchCV(multinomial_naive_bayes, param_grid, cv=5, scoring='accuracy')
grid_search.fit(X_train_chi2, y_train)

# Best parameters and score
print("Best Parameters:", grid_search.best_params_)
print("Best Cross-Validation Score:", grid_search.best_score_)

# Evaluate on test set
test_score = grid_search.score(X_test_chi2, y_test)
print("Test Set Score:", test_score)

Best Parameters: {'alpha': 0.1, 'fit_prior': True}
Best Cross-Validation Score: 0.9390625
Test Set Score: 0.83125


In [8]:
# fit the model
multinomial_naive_bayes = MultinomialNB( alpha = 0.1, fit_prior = True)
multinomial_naive_bayes.fit(X_train_chi2, y_train)
# predictions
y_pred = multinomial_naive_bayes.predict(X_test_chi2)
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')
f1 = f1_score(y_test, y_pred, average='weighted')

print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")

Accuracy: 0.8313
Precision: 0.8376
Recall: 0.8313
F1 Score: 0.8304


### Feature importance

In [9]:
# Get log probabilities
log_probs = multinomial_naive_bayes.feature_log_prob_

feature_names = chi2_selector.get_feature_names_out()

# Calculate log-odds ratio (Class 1 log-probs minus Class 0 log-probs)
log_odds = log_probs[1, :] - log_probs[0, :]  # Class 1 (Genuine) vs Class 0 (Deceptive)

# Convert to DataFrame
log_odds_df = pd.DataFrame(log_odds, index=feature_names, columns=['log_odds'])

# Sort by log-odds
deceptive_features = log_odds_df.sort_values(by='log_odds', ascending=True).head()  # More negative log-odds -> Deceptive
genuine_features = log_odds_df.sort_values(by='log_odds', ascending=False).head()  # More positive log-odds -> Genuine

# Print the top features for each class based on log-odds
print("Top features for Deceptive Reviews:")
print(deceptive_features)

print("\nTop features for Genuine Reviews:")
print(genuine_features)

Top features for Deceptive Reviews:
       log_odds
x4901 -4.991379
x4195 -4.838409
x5357 -4.657739
x1986 -4.553478
x5791 -4.437068

Top features for Genuine Reviews:
       log_odds
x4597  5.209655
x5614  4.832579
x2483  4.753172
x5774  4.753172
x5713  4.468241


## Logistic Regression

### Train and accuracy

In [10]:
# parameters
param_grid = {
    'C': [0.001, 0.01, 0.1, 1.0, 5, 10]
}

logistic_regression = LogisticRegression(penalty='l1', solver='liblinear')

# Set up GridSearchCV
grid_search = GridSearchCV(logistic_regression, param_grid, cv=10, scoring='accuracy')
grid_search.fit(X_train2, y_train)

# Best parameters and score
print("Best Parameters:", grid_search.best_params_)
print("Best Cross-Validation Score:", grid_search.best_score_)

# Evaluate on test set
test_score = grid_search.score(X_test2, y_test)
print("Test Set Score:", test_score)

Best Parameters: {'C': 10}
Best Cross-Validation Score: 0.8046875
Test Set Score: 0.75625


In [11]:
# fit the model
logistic_regression = LogisticRegression(penalty='l1', solver='liblinear', C=5)
logistic_regression.fit(X_train2, y_train)
# predictions
y_pred = logistic_regression.predict(X_test2)
accuracy = accuracy_score(y_test ,y_pred)
precision = precision_score(y_test ,y_pred,average='binary')
recall = recall_score(y_test ,y_pred,average='binary')
f1 = f1_score(y_test, y_pred, average='binary')

print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")

Accuracy: 0.7750
Precision: 0.7821
Recall: 0.7625
F1 Score: 0.7722


In [12]:
# Check number of non-zero coefficients
non_zero_coefficients = (logistic_regression.coef_ != 0).sum()
total_coefficients = logistic_regression.coef_.size
print(f"Non-zero coefficients: {non_zero_coefficients}/{total_coefficients}")


Non-zero coefficients: 187/6933


### Feature importance

In [13]:
# Get the coefficients
coefficients = logistic_regression.coef_[0]
feature_names = vectorizer.get_feature_names_out()

# Create a DataFrame for better visualization
coef_df = pd.DataFrame({'feature': feature_names, 'coefficient': coefficients})

# Separate features for genuine and deceptive reviews
genuine_features = coef_df[coef_df['coefficient'] > 0].sort_values(by='coefficient', ascending=False)
deceptive_features = coef_df[coef_df['coefficient'] < 0].sort_values(by='coefficient')

# Display the results
print("Important Features for Genuine Reviews:")
print(genuine_features.head())

print("\nImportant Features for Deceptive Reviews:")
print(deceptive_features.head())

Important Features for Genuine Reviews:
       feature  coefficient
5755      star    11.102971
4766      rate     9.272784
5296  security     9.227650
2671     great     8.016123
3530  location     7.857808

Important Features for Deceptive Reviews:
         feature  coefficient
3605      luxury   -13.657444
2321     finally   -13.032198
3803  millennium   -12.447030
4820    recently   -12.203763
1561     decided   -11.552617


## Classification Trees

### Train and accuracy

In [14]:
# parameters
param_grid = {
    'criterion': ['gini', 'entropy'],  # Function to measure the quality of a split
    'max_depth': [2, 4, 6, 8, 10],  # Maximum depth of the tree
    'min_samples_split': [2, 5, 10],  # Minimum number of samples required to split an internal node
    'min_samples_leaf': [1, 2, 4, 6],  # Minimum number of samples required to be at a leaf node
    'ccp_alpha': np.linspace(0, 0.1, 11) 
}


decision_tree = DecisionTreeClassifier(criterion='gini', max_depth=None, random_state=42)

# Set up GridSearchCV
grid_search = GridSearchCV(decision_tree, param_grid, cv=5, scoring='accuracy')
grid_search.fit(X_train, y_train)

# Best parameters and score
print("Best Parameters:", grid_search.best_params_)
print("Best Cross-Validation Score:", grid_search.best_score_)

# Evaluate on test set
test_score = grid_search.score(X_test, y_test)
print("Test Set Score:", test_score)

Best Parameters: {'ccp_alpha': np.float64(0.01), 'criterion': 'gini', 'max_depth': 10, 'min_samples_leaf': 1, 'min_samples_split': 2}
Best Cross-Validation Score: 0.640625
Test Set Score: 0.6625


In [15]:
# fit the model
decision_tree = DecisionTreeClassifier(ccp_alpha= 0.01, criterion= 'gini', max_depth= 10, min_samples_leaf= 1, min_samples_split= 2, random_state=42)
decision_tree.fit(X_train, y_train)
# predictions
y_pred = decision_tree.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='binary')
recall = recall_score(y_test, y_pred, average='binary')
f1 = f1_score(y_test, y_pred, average='binary')

print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")

Accuracy: 0.6625
Precision: 0.6275
Recall: 0.8000
F1 Score: 0.7033


### Feature Importance

In [16]:
X_train_arr = X_train.toarray() 
X_test_arr = X_test.toarray()

perm_importance = permutation_importance(decision_tree, X_test_arr, y_test, n_repeats=10, random_state=42)

feature_names = vectorizer.get_feature_names_out()
# Display the importance scores for the positive class
importance_df = pd.DataFrame({
    'Feature': feature_names,
    'Importance': perm_importance.importances_mean
}).sort_values(by='Importance', ascending=False)

print("\nPermutation Importance:")
print(importance_df.head())


Permutation Importance:
         Feature  Importance
5302      seemed    0.059375
3605      luxury    0.056250
2175  experience    0.036875
1561     decided    0.033125
3530    location    0.032500


## Random Forest

### Train and accuracy

In [17]:

# parameters
param_grid = {
    'n_estimators': [50, 100],  # Number of trees in the forest
    'max_features': [ 'sqrt', 'log2'],  # Number of features to consider for the best split
    'criterion': ['gini', 'entropy'],
    'max_depth': [3, 7, 10, 20],  # Maximum depth of the tree
    'min_samples_split': [2, 5],  # Minimum number of samples required to split an internal node
    'min_samples_leaf': [1, 2]  # Minimum number of samples required to be at a leaf node
}


# Store results
results = []

# Loop through hyperparameter combinations
for params in ParameterGrid(param_grid):
    model = RandomForestClassifier(oob_score=True, random_state=42, **params)
    model.fit(X_train, y_train)
    
    # Record OOB score
    oob_score = model.oob_score_
    results.append({**params, 'oob_score': oob_score})

# Convert results to DataFrame for easier analysis
results_df = pd.DataFrame(results)

# Find the best hyperparameters based on OOB score
best_params = results_df.loc[results_df['oob_score'].idxmax()]
print("Best Hyperparameters:\n", best_params)

Best Hyperparameters:
 criterion               gini
max_depth                 20
max_features            sqrt
min_samples_leaf           2
min_samples_split          5
n_estimators             100
oob_score            0.80625
Name: 55, dtype: object


In [18]:
# fit the model
random_forest = RandomForestClassifier(criterion = 'gini', max_depth = 20, max_features="sqrt", min_samples_leaf=2, min_samples_split=5, n_estimators=100, random_state = 42)
random_forest.fit(X_train, y_train)
# predictions
y_pred = random_forest.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average = 'binary')
recall = recall_score(y_test, y_pred, average = 'binary')
f1 = f1_score(y_test, y_pred,average = 'binary')

print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")

Accuracy: 0.7812
Precision: 0.7778
Recall: 0.7875
F1 Score: 0.7826


### Feature Importance

In [19]:
perm_importance = permutation_importance(random_forest, X_test_arr, y_test, n_repeats=10, random_state=42)

feature_names = vectorizer.get_feature_names_out()
# Display the importance scores for the positive class
importance_df = pd.DataFrame({
    'Feature': feature_names,
    'Importance': perm_importance.importances_mean
}).sort_values(by='Importance', ascending=False)

print("\nPermutation Importance:")
print(importance_df.head())


Permutation Importance:
       Feature  Importance
4820  recently    0.018750
3605    luxury    0.018125
3530  location    0.015000
6210      tiny    0.012500
5786    stayed    0.010000


# Bigram

## Dataset Preprocessing

In [20]:
traindf = pd.read_csv("train_reviews.csv")
trainsentences = traindf["Review"].values
trainlabels = traindf["Label"].values
testdf = pd.read_csv("test_reviews.csv")
testsentences = testdf["Review"].values
testlabels = testdf["Label"].values 

with open("english", "r") as file:
    stop_words = set(file.read().splitlines())


def preprocess_text(text):
    text = text.translate(str.maketrans('', '', string.punctuation))
    text = text.lower()
    text = re.sub(r'\d+', '', text) 
    words = text.split()   
    words = [word for word in words if word not in stop_words]    
    text = ' '.join(words).strip()
    
    return text

train_sentences_preprocessed = [preprocess_text(sentence) for sentence in trainsentences]
test_sentences_preprocessed = [preprocess_text(sentence) for sentence in testsentences]
train_df_preprocessed = pd.DataFrame({'Review': train_sentences_preprocessed,'Label': trainlabels})
test_df_preprocessed = pd.DataFrame({'Review': test_sentences_preprocessed,'Label': testlabels})

In [21]:
trainsentences = train_df_preprocessed["Review"].values
y_train = train_df_preprocessed["Label"].values

testsentences = test_df_preprocessed["Review"].values
y_test = test_df_preprocessed["Label"].values

# Encode labels as integers
le = LabelEncoder()
y_train = le.fit_transform(y_train)
y_test = le.transform(y_test)

# vectorizing
vectorizer = CountVectorizer(ngram_range=(1,2)) # ngram range for specifying unigrams and bigrams (1,1) - unigram, (2,2) - bigram, (1,2) - both
X_train = vectorizer.fit_transform(trainsentences)
X_test = vectorizer.transform(testsentences)

## Multinomial naive Bayes

### Train and accuracy without feature selection

In [22]:
# parameters
param_grid = {
    'alpha': [0.1, 0.5, 1.0, 1.5, 2.0],  # Laplace smoothing parameter
    'fit_prior': [True, False]
}
multinomial_naive_bayes = MultinomialNB()

# Set up GridSearchCV
grid_search = GridSearchCV(multinomial_naive_bayes, param_grid, cv=5, scoring='accuracy')
grid_search.fit(X_train, y_train)

# Best parameters and score
print("Best Parameters:", grid_search.best_params_)
print("Best Cross-Validation Score:", grid_search.best_score_)

# Evaluate on test set
test_score = grid_search.score(X_test, y_test)
print("Test Set Score:", test_score)

Best Parameters: {'alpha': 1.0, 'fit_prior': True}
Best Cross-Validation Score: 0.828125
Test Set Score: 0.86875


In [23]:
# fit the model
multinomial_naive_bayes = MultinomialNB( alpha = 0.1, fit_prior = True)
multinomial_naive_bayes.fit(X_train, y_train)
# predictions
y_pred = multinomial_naive_bayes.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='binary')
recall = recall_score(y_test, y_pred, average='binary')
f1 = f1_score(y_test, y_pred, average='binary')

print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")

Accuracy: 0.8438
Precision: 0.8667
Recall: 0.8125
F1 Score: 0.8387


### Train and accuracy with chi squared test for feature selection

In [24]:
# Perform Chi-squared feature selection
chi2_selector = SelectKBest(chi2, k=2000) #2000 features perform best
X_train_chi2 = chi2_selector.fit_transform(X_train, y_train)
X_test_chi2 = chi2_selector.transform(X_test)

# Train a Multinomial Naive Bayes model
nb_model = MultinomialNB()
nb_model.fit(X_train_chi2, y_train)

# Predict on the test set
y_pred = nb_model.predict(X_test_chi2)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)

# Get the feature names for the selected features
selected_feature_indices = chi2_selector.get_support(indices=True)
selected_feature_names = vectorizer.get_feature_names_out()[selected_feature_indices]
precision = precision_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')
f1 = f1_score(y_test, y_pred, average='weighted')

print(f"Selected Features: {selected_feature_names}")
print(f"Test Set Accuracy: {accuracy}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")

Selected Features: ['able relax' 'accept' 'accepted' ... 'youre going' 'youre traveler'
 'zone']
Test Set Accuracy: 0.86875
Precision: 0.8688
Recall: 0.8688
F1 Score: 0.8687


In [25]:
# parameters
param_grid = {
    'alpha': [0.1, 0.5, 1.0, 1.5, 2.0],  # Laplace smoothing parameter
    'fit_prior': [True, False]
}
multinomial_naive_bayes = MultinomialNB()

# Set up GridSearchCV
grid_search = GridSearchCV(multinomial_naive_bayes, param_grid, cv=5, scoring='accuracy')
grid_search.fit(X_train_chi2, y_train)

# Best parameters and score
print("Best Parameters:", grid_search.best_params_)
print("Best Cross-Validation Score:", grid_search.best_score_)

# Evaluate on test set
test_score = grid_search.score(X_test_chi2, y_test)
print("Test Set Score:", test_score)

Best Parameters: {'alpha': 0.1, 'fit_prior': True}
Best Cross-Validation Score: 0.9765625
Test Set Score: 0.83125


In [26]:
# fit the model
multinomial_naive_bayes = MultinomialNB( alpha = 0.1, fit_prior = True)
multinomial_naive_bayes.fit(X_train_chi2, y_train)
# predictions
y_pred = multinomial_naive_bayes.predict(X_test_chi2)
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')
f1 = f1_score(y_test, y_pred, average='weighted')

print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")

Accuracy: 0.8313
Precision: 0.8325
Recall: 0.8313
F1 Score: 0.8311


### Feature importance

In [27]:
# Get log probabilities
log_probs = multinomial_naive_bayes.feature_log_prob_

feature_names = chi2_selector.get_feature_names_out()

# Calculate log-odds ratio (Class 1 log-probs minus Class 0 log-probs)
log_odds = log_probs[1, :] - log_probs[0, :]  # Class 1 (Genuine) vs Class 0 (Deceptive)

# Convert to DataFrame
log_odds_df = pd.DataFrame(log_odds, index=feature_names, columns=['log_odds'])

# Sort by log-odds
deceptive_features = log_odds_df.sort_values(by='log_odds', ascending=True).head()  # More negative log-odds -> Deceptive
genuine_features = log_odds_df.sort_values(by='log_odds', ascending=False).head()  # More positive log-odds -> Genuine

# Print the top features for each class based on log-odds
print("Top features for Deceptive Reviews:")
print(deceptive_features)

print("\nTop features for Genuine Reviews:")
print(genuine_features)

Top features for Deceptive Reviews:
        log_odds
x7336  -5.298319
x38165 -5.198735
x26628 -4.963741
x25331 -4.963741
x20733 -4.963741

Top features for Genuine Reviews:
        log_odds
x32420  5.305812
x4829   4.928736
x39540  4.928736
x40756  4.849329
x16913  4.849329


## Logistic Regression

### Train and accuracy

In [28]:
# parameters
param_grid = {
    'C': [0.001, 0.01, 0.1, 1.0, 5, 10]
}

logistic_regression = LogisticRegression(penalty='l1', solver='liblinear')

# Set up GridSearchCV
grid_search = GridSearchCV(logistic_regression, param_grid, cv=10, scoring='accuracy')
grid_search.fit(X_train, y_train)

# Best parameters and score
print("Best Parameters:", grid_search.best_params_)
print("Best Cross-Validation Score:", grid_search.best_score_)

# Evaluate on test set
test_score = grid_search.score(X_test, y_test)
print("Test Set Score:", test_score)

Best Parameters: {'C': 5}
Best Cross-Validation Score: 0.8125
Test Set Score: 0.7875


In [29]:
# fit the model
logistic_regression = LogisticRegression(penalty='l1', solver='liblinear', C=10)
logistic_regression.fit(X_train, y_train)
# predictions
y_pred = logistic_regression.predict(X_test)
accuracy = accuracy_score(y_test ,y_pred)
precision = precision_score(y_test ,y_pred,average='binary')
recall = recall_score(y_test ,y_pred,average='binary')
f1 = f1_score(y_test, y_pred, average='binary')

print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")

Accuracy: 0.7937
Precision: 0.7901
Recall: 0.8000
F1 Score: 0.7950


In [30]:
# Check number of non-zero coefficients
non_zero_coefficients = (logistic_regression.coef_ != 0).sum()
total_coefficients = logistic_regression.coef_.size
print(f"Non-zero coefficients: {non_zero_coefficients}/{total_coefficients}")


Non-zero coefficients: 277/49391


### Feature importance

In [31]:
# Get the coefficients
coefficients = logistic_regression.coef_[0]
feature_names = vectorizer.get_feature_names_out()

# Create a DataFrame for better visualization
coef_df = pd.DataFrame({'feature': feature_names, 'coefficient': coefficients})

# Separate features for genuine and deceptive reviews
genuine_features = coef_df[coef_df['coefficient'] > 0].sort_values(by='coefficient', ascending=False)
deceptive_features = coef_df[coef_df['coefficient'] < 0].sort_values(by='coefficient')

# Display the results
print("Important Features for Genuine Reviews:")
print(genuine_features.head())

print("\nImportant Features for Deceptive Reviews:")
print(deceptive_features.head())

Important Features for Genuine Reviews:
            feature  coefficient
40613          star     3.250420
48684         world     2.995086
4829   booked hotel     2.665454
5996           cant     2.554130
44818           try     2.290418

Important Features for Deceptive Reviews:
             feature  coefficient
34194          relax    -2.958042
20245  hotel chicago    -2.710557
33805         recent    -2.656534
15648        finally    -2.455652
20464  hotel located    -2.440066


## Classification Trees

### Train and accuracy

In [32]:
# parameters
param_grid = {
    'criterion': ['gini', 'entropy'],  # Function to measure the quality of a split
    'max_depth': [ 2, 4, 6, 8, 10],  # Maximum depth of the tree
    'min_samples_split': [2, 5, 10],  # Minimum number of samples required to split an internal node
    'min_samples_leaf': [1, 2, 4, 6],  # Minimum number of samples required to be at a leaf node
    'ccp_alpha': np.linspace(0, 0.1, 11) 
}


decision_tree = DecisionTreeClassifier(criterion='gini', max_depth=None, random_state=42)

# Set up GridSearchCV
grid_search = GridSearchCV(decision_tree, param_grid, cv=10, scoring='accuracy')
grid_search.fit(X_train, y_train)

# Best parameters and score
print("Best Parameters:", grid_search.best_params_)
print("Best Cross-Validation Score:", grid_search.best_score_)

# Evaluate on test set
test_score = grid_search.score(X_test, y_test)
print("Test Set Score:", test_score)

Best Parameters: {'ccp_alpha': np.float64(0.0), 'criterion': 'gini', 'max_depth': 8, 'min_samples_leaf': 1, 'min_samples_split': 5}
Best Cross-Validation Score: 0.7109375
Test Set Score: 0.65


In [33]:
# fit the model
decision_tree = DecisionTreeClassifier(ccp_alpha= 0.0, criterion= 'gini', max_depth= 8, min_samples_leaf= 6, min_samples_split= 2, random_state=42)
decision_tree.fit(X_train, y_train)
# predictions
y_pred = decision_tree.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='binary')
recall = recall_score(y_test, y_pred, average='binary')
f1 = f1_score(y_test, y_pred, average='binary')

print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")

Accuracy: 0.6312
Precision: 0.6154
Recall: 0.7000
F1 Score: 0.6550


### Feature Importance

In [ ]:
X_train_arr = X_train.toarray() 
X_test_arr = X_test.toarray()

perm_importance = permutation_importance(decision_tree, X_test_arr, y_test, n_repeats=10, random_state=42)

feature_names = vectorizer.get_feature_names_out()
# Display the importance scores for the positive class
importance_df = pd.DataFrame({
    'Feature': feature_names,
    'Importance': perm_importance.importances_mean
}).sort_values(by='Importance', ascending=False)

print("\nPermutation Importance:")
print(importance_df.head())

## Random Forest

### Train and accuracy

In [ ]:

# parameters
param_grid = {
    'n_estimators': [50, 100],  # Number of trees in the forest
    'max_features': [ 'sqrt', 'log2'],  # Number of features to consider for the best split
    'criterion': ['gini', 'entropy'],
    'max_depth': [3, 7, 10, 20],  # Maximum depth of the tree
    'min_samples_split': [2, 5],  # Minimum number of samples required to split an internal node
    'min_samples_leaf': [1, 2]  # Minimum number of samples required to be at a leaf node
}


# Store results
results = []

# Loop through hyperparameter combinations
for params in ParameterGrid(param_grid):
    model = RandomForestClassifier(oob_score=True, random_state=42, **params)
    model.fit(X_train, y_train)
    
    # Record OOB score
    oob_score = model.oob_score_
    results.append({**params, 'oob_score': oob_score})

# Convert results to DataFrame for easier analysis
results_df = pd.DataFrame(results)

# Find the best hyperparameters based on OOB score
best_params = results_df.loc[results_df['oob_score'].idxmax()]
print("Best Hyperparameters:\n", best_params)

Best Hyperparameters:
 criterion            entropy
max_depth                NaN
max_features            sqrt
min_samples_leaf           2
min_samples_split          2
n_estimators             100
oob_score             0.7875
Name: 53, dtype: object


In [ ]:
# fit the model
random_forest = RandomForestClassifier(criterion = 'entropy', max_depth = None, max_features="sqrt", min_samples_leaf=2, min_samples_split=2, n_estimators=100, random_state = 42)
random_forest.fit(X_train, y_train)
# predictions
y_pred = random_forest.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average = 'binary')
recall = recall_score(y_test, y_pred, average = 'binary')
f1 = f1_score(y_test, y_pred,average = 'binary')

print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")

Accuracy: 0.7125
Precision: 0.6466
Recall: 0.9375
F1 Score: 0.7653


### Feature Importance

In [ ]:
perm_importance = permutation_importance(random_forest, X_test_arr, y_test, n_repeats=10, random_state=42)

feature_names = vectorizer.get_feature_names_out()
# Display the importance scores for the positive class
importance_df = pd.DataFrame({
    'Feature': feature_names,
    'Importance': perm_importance.importances_mean
}).sort_values(by='Importance', ascending=False)

print("\nPermutation Importance:")
print(importance_df.head())

KeyboardInterrupt: 